In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Set plotting style for professional presentations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Step 1: Load and Explore Flight Data

In [ ]:
# Load sample flight data (would be real FDR data in practice)
# For demo purposes, we'll generate realistic flight data

def generate_flight_data():
    """Generate realistic flight profile data for demonstration"""
    
    # Flight phases: taxi, takeoff, climb, cruise, descent, approach, landing
    time_points = np.linspace(0, 120, 1200)  # 2-hour flight, 6-second intervals
    
    # Altitude profile (ft)
    altitude = np.zeros_like(time_points)
    
    # Taxi (0-5 min): ground level
    altitude[time_points <= 5] = 50 + np.random.normal(0, 5, len(altitude[time_points <= 5]))
    
    # Takeoff and climb (5-25 min): 0 to 35,000 ft
    climb_mask = (time_points > 5) & (time_points <= 25)
    climb_time = time_points[climb_mask] - 5
    altitude[climb_mask] = 50 + 35000 * (1 - np.exp(-climb_time/8)) + np.random.normal(0, 100, len(climb_time))
    
    # Cruise (25-90 min): ~35,000 ft with minor variations
    cruise_mask = (time_points > 25) & (time_points <= 90)
    altitude[cruise_mask] = 35000 + np.random.normal(0, 200, len(altitude[cruise_mask]))
    
    # Descent (90-115 min): 35,000 to 1000 ft
    descent_mask = (time_points > 90) & (time_points <= 115)
    descent_time = time_points[descent_mask] - 90
    altitude[descent_mask] = 35000 * np.exp(-descent_time/10) + 1000 + np.random.normal(0, 150, len(descent_time))
    
    # Landing (115-120 min): approach to ground
    landing_mask = time_points > 115
    landing_time = time_points[landing_mask] - 115
    altitude[landing_mask] = 1000 * np.exp(-landing_time*2) + 50 + np.random.normal(0, 20, len(landing_time))
    
    # Airspeed (knots) - varies with flight phase
    airspeed = np.zeros_like(time_points)
    airspeed[time_points <= 5] = 20 + np.random.normal(0, 5, len(airspeed[time_points <= 5]))  # Taxi
    airspeed[climb_mask] = 250 + 200 * (climb_time / 20) + np.random.normal(0, 10, len(climb_time))  # Climb
    airspeed[cruise_mask] = 450 + np.random.normal(0, 15, len(airspeed[cruise_mask]))  # Cruise
    airspeed[descent_mask] = 450 - 200 * (descent_time / 25) + np.random.normal(0, 12, len(descent_time))  # Descent
    airspeed[landing_mask] = 180 - 130 * (landing_time / 5) + np.random.normal(0, 8, len(landing_time))  # Landing
    
    # Vertical speed (ft/min)
    vertical_speed = np.gradient(altitude) * 10  # Approximate derivative
    
    # Engine parameters
    n1_speed = 70 + 25 * (airspeed / 450) + np.random.normal(0, 3, len(time_points))
    fuel_flow = 1000 + 500 * (airspeed / 450) + np.random.normal(0, 50, len(time_points))
    
    # Create DataFrame
    data = {
        'time_minutes': time_points,
        'altitude': altitude,
        'airspeed': airspeed,
        'vertical_speed': vertical_speed,
        'n1_speed': n1_speed,
        'fuel_flow': fuel_flow
    }
    
    return pd.DataFrame(data)

# Generate our demo flight data
flight_data = generate_flight_data()

print("Flight Data Overview:")
print(f"Duration: {flight_data['time_minutes'].max():.1f} minutes")
print(f"Max Altitude: {flight_data['altitude'].max():.0f} ft")
print(f"Max Airspeed: {flight_data['airspeed'].max():.0f} knots")
print("\nFirst 5 rows:")
flight_data.head()

## Step 2: Traditional Flight Profile Visualization

In [ ]:
# Create comprehensive flight profile visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Boeing 737 Flight Profile Analysis', fontsize=16, fontweight='bold')

# Altitude vs Time
ax1.plot(flight_data['time_minutes'], flight_data['altitude'], 'b-', linewidth=2)
ax1.set_title('Altitude Profile', fontweight='bold')
ax1.set_xlabel('Time (minutes)')
ax1.set_ylabel('Altitude (ft)')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, None)

# Airspeed vs Time
ax2.plot(flight_data['time_minutes'], flight_data['airspeed'], 'r-', linewidth=2)
ax2.set_title('Airspeed Profile', fontweight='bold')
ax2.set_xlabel('Time (minutes)')
ax2.set_ylabel('Airspeed (knots)')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, None)

# Vertical Speed vs Time
ax3.plot(flight_data['time_minutes'], flight_data['vertical_speed'], 'g-', linewidth=2)
ax3.axhline(y=0, color='k', linestyle='--', alpha=0.5)
ax3.set_title('Vertical Speed Profile', fontweight='bold')
ax3.set_xlabel('Time (minutes)')
ax3.set_ylabel('Vertical Speed (ft/min)')
ax3.grid(True, alpha=0.3)

# 3D Trajectory (Altitude vs Airspeed vs Time)
scatter = ax4.scatter(flight_data['airspeed'], flight_data['altitude'], 
                     c=flight_data['time_minutes'], cmap='viridis', alpha=0.6, s=20)
ax4.set_title('Airspeed vs Altitude (colored by time)', fontweight='bold')
ax4.set_xlabel('Airspeed (knots)')
ax4.set_ylabel('Altitude (ft)')
ax4.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax4, label='Time (minutes)')

plt.tight_layout()
plt.show()

print("\n📊 Discussion Points:")
print("• Can you identify the major flight phases visually?")
print("• What patterns do you notice in the relationships between parameters?")
print("• How might a pilot or air traffic controller use this information?")

## Step 3: Machine Learning Approach - Automatic Flight Phase Detection

**Challenge**: Can we automatically identify flight phases without manual labeling?

**Approach**: Use unsupervised learning (K-means clustering)

In [ ]:
# Prepare features for machine learning
# Select relevant features for flight phase identification
feature_columns = ['altitude', 'airspeed', 'vertical_speed']
X = flight_data[feature_columns].values

# Standardize features (important for clustering algorithms)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Feature Matrix Shape:", X_scaled.shape)
print("Features used:", feature_columns)
print("\nFeature Statistics (after scaling):")
print(f"Mean: {X_scaled.mean(axis=0)}")
print(f"Std:  {X_scaled.std(axis=0)}")

In [ ]:
# Apply K-means clustering to identify flight phases
# We expect about 5-6 major phases: taxi, climb, cruise, descent, approach, landing
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
flight_phases = kmeans.fit_predict(X_scaled)

# Add predicted phases to our dataframe
flight_data['ml_phase'] = flight_phases

# Analyze the discovered phases
print("Machine Learning Flight Phase Analysis:")
print("="*50)

phase_summary = flight_data.groupby('ml_phase').agg({
    'time_minutes': ['min', 'max', 'count'],
    'altitude': ['mean', 'std'],
    'airspeed': ['mean', 'std'],
    'vertical_speed': ['mean', 'std']
}).round(1)

print("\nPhase Summary:")
for phase in range(n_clusters):
    phase_data = flight_data[flight_data['ml_phase'] == phase]
    duration = phase_data['time_minutes'].max() - phase_data['time_minutes'].min()
    print(f"\nPhase {phase}:")
    print(f"  Duration: {duration:.1f} minutes")
    print(f"  Altitude: {phase_data['altitude'].mean():.0f} ± {phase_data['altitude'].std():.0f} ft")
    print(f"  Airspeed: {phase_data['airspeed'].mean():.0f} ± {phase_data['airspeed'].std():.0f} knots")
    print(f"  Vertical Speed: {phase_data['vertical_speed'].mean():.0f} ± {phase_data['vertical_speed'].std():.0f} ft/min")

In [ ]:
# Visualize ML-discovered flight phases
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Machine Learning Flight Phase Detection Results', fontsize=16, fontweight='bold')

# Color map for consistent phase colors
colors = plt.cm.Set1(np.linspace(0, 1, n_clusters))

# Time-series with colored phases
for phase in range(n_clusters):
    phase_mask = flight_data['ml_phase'] == phase
    ax1.scatter(flight_data[phase_mask]['time_minutes'], 
               flight_data[phase_mask]['altitude'],
               c=[colors[phase]], label=f'Phase {phase}', alpha=0.7, s=15)
ax1.set_title('Altitude vs Time (ML Phases)', fontweight='bold')
ax1.set_xlabel('Time (minutes)')
ax1.set_ylabel('Altitude (ft)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2D feature space visualization
for phase in range(n_clusters):
    phase_mask = flight_data['ml_phase'] == phase
    ax2.scatter(flight_data[phase_mask]['airspeed'], 
               flight_data[phase_mask]['altitude'],
               c=[colors[phase]], label=f'Phase {phase}', alpha=0.7, s=15)
ax2.set_title('Airspeed vs Altitude (ML Phases)', fontweight='bold')
ax2.set_xlabel('Airspeed (knots)')
ax2.set_ylabel('Altitude (ft)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3D visualization using PCA for dimensionality reduction
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

for phase in range(n_clusters):
    phase_mask = flight_data['ml_phase'] == phase
    ax3.scatter(X_pca[phase_mask, 0], X_pca[phase_mask, 1],
               c=[colors[phase]], label=f'Phase {phase}', alpha=0.7, s=15)
ax3.set_title('PCA Visualization of Flight Phases', fontweight='bold')
ax3.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
ax3.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Phase duration analysis
phase_durations = []
phase_names = []
for phase in range(n_clusters):
    phase_data = flight_data[flight_data['ml_phase'] == phase]
    duration = phase_data['time_minutes'].max() - phase_data['time_minutes'].min()
    phase_durations.append(duration)
    phase_names.append(f'Phase {phase}')

ax4.bar(phase_names, phase_durations, color=colors)
ax4.set_title('Phase Duration Analysis', fontweight='bold')
ax4.set_xlabel('Flight Phase')
ax4.set_ylabel('Duration (minutes)')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🤖 ML Performance:")
print(f"• Total variance explained by PCA: {pca.explained_variance_ratio_.sum():.1%}")
print(f"• Number of phases detected: {n_clusters}")
print(f"• Average silhouette score: {silhouette_score(X_scaled, flight_phases):.3f}")

In [ ]:
# Import silhouette score for cluster quality assessment
from sklearn.metrics import silhouette_score, silhouette_samples

# Evaluate clustering quality
sil_score = silhouette_score(X_scaled, flight_phases)
print(f"Overall Silhouette Score: {sil_score:.3f}")
print("(Closer to 1.0 is better, >0.5 is considered good)")

# Individual silhouette scores for each sample
sample_silhouette_values = silhouette_samples(X_scaled, flight_phases)

# Silhouette analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Silhouette plot
y_lower = 10
for i in range(n_clusters):
    cluster_silhouette_values = sample_silhouette_values[flight_phases == i]
    cluster_silhouette_values.sort()
    
    size_cluster_i = cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    
    ax1.fill_betweenx(np.arange(y_lower, y_upper),
                      0, cluster_silhouette_values,
                      facecolor=colors[i], edgecolor=colors[i], alpha=0.7)
    
    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

ax1.set_xlabel('Silhouette Coefficient Values')
ax1.set_ylabel('Cluster Label')
ax1.set_title('Silhouette Plot for Flight Phase Clustering')
ax1.axvline(x=sil_score, color="red", linestyle="--", label=f'Average Score: {sil_score:.3f}')
ax1.legend()

# Cluster centers in original feature space
centers = scaler.inverse_transform(kmeans.cluster_centers_)
centers_df = pd.DataFrame(centers, columns=feature_columns)
centers_df.index = [f'Phase {i}' for i in range(n_clusters)]

# Plot cluster centers
sns.heatmap(centers_df.T, annot=True, fmt='.0f', cmap='viridis', ax=ax2)
ax2.set_title('Flight Phase Characteristics (Cluster Centers)')
ax2.set_xlabel('Flight Phase')

plt.tight_layout()
plt.show()

print("\nCluster Centers (Original Units):")
print(centers_df)

## Step 4: Discussion & Real-World Applications

### What We Just Accomplished
1. **Automated Pattern Recognition**: ML identified distinct flight phases without manual labeling
2. **Multi-dimensional Analysis**: Considered multiple aircraft parameters simultaneously  
3. **Quantitative Assessment**: Used silhouette analysis to evaluate cluster quality
4. **Visualization**: Created interpretable plots for aviation professionals

### Real-World Applications

In [ ]:
# Demonstrate practical applications
print("🛩️ PRACTICAL AEROSPACE APPLICATIONS:")
print("="*60)

applications = [
    {
        'domain': 'Flight Operations',
        'application': 'Automatic flight phase detection for pilot training analysis',
        'benefit': 'Identify non-standard procedures, optimize training programs'
    },
    {
        'domain': 'Predictive Maintenance', 
        'application': 'Engine health monitoring using flight phase context',
        'benefit': 'Phase-specific wear patterns, targeted maintenance scheduling'
    },
    {
        'domain': 'Fuel Optimization',
        'application': 'Analyze fuel consumption patterns across flight phases', 
        'benefit': 'Optimize climb/descent profiles, reduce operational costs'
    },
    {
        'domain': 'Safety Analysis',
        'application': 'Anomaly detection during critical flight phases',
        'benefit': 'Early warning systems, accident prevention'
    },
    {
        'domain': 'Air Traffic Management',
        'application': 'Predict aircraft behavior for traffic flow optimization',
        'benefit': 'Reduce delays, improve airspace efficiency'
    }
]

for i, app in enumerate(applications, 1):
    print(f"\n{i}. {app['domain'].upper()}")
    print(f"   Application: {app['application']}")
    print(f"   Benefit: {app['benefit']}")

print("\n" + "="*60)
print("💡 NEXT STEPS IN THIS COURSE:")
print("• Week 2: Build regression models for aircraft performance prediction")
print("• Week 4: Classification algorithms for fault detection")
print("• Week 10: Time series models for trajectory prediction")
print("• Week 11: Physics-informed ML for aerodynamic modeling")

## Step 5: Interactive Exercise

**Your Turn**: Modify the analysis to answer these questions:

In [ ]:
# EXERCISE 1: Try different numbers of clusters
print("EXERCISE 1: Optimal Number of Clusters")
print("Try changing n_clusters above to 3, 4, 6, or 7")
print("Question: What happens to the silhouette score? Which number seems most appropriate?")
print()

# EXERCISE 2: Feature importance analysis
print("EXERCISE 2: Feature Selection")
print("Try removing one feature at a time from feature_columns")
print("Question: Which features are most important for phase detection?")
print()

# EXERCISE 3: Add new features
print("EXERCISE 3: Additional Features")
print("Add 'n1_speed' and 'fuel_flow' to the feature list")
print("Question: Do engine parameters improve phase detection?")
print()

# Quick experiment framework
def experiment_clusters(n_clusters_range, features):
    """Test different cluster numbers and return silhouette scores"""
    X_exp = flight_data[features].values
    X_exp_scaled = StandardScaler().fit_transform(X_exp)
    
    scores = []
    for n in n_clusters_range:
        kmeans = KMeans(n_clusters=n, random_state=42)
        labels = kmeans.fit_predict(X_exp_scaled)
        score = silhouette_score(X_exp_scaled, labels)
        scores.append(score)
        print(f"Clusters: {n}, Silhouette Score: {score:.3f}")
    
    return scores

# Example: Test cluster numbers 2-8
print("\nTesting different cluster numbers:")
cluster_range = range(2, 9)
scores = experiment_clusters(cluster_range, ['altitude', 'airspeed', 'vertical_speed'])

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, scores, 'bo-', linewidth=2, markersize=8)
plt.title('Silhouette Score vs Number of Clusters', fontweight='bold')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.grid(True, alpha=0.3)
plt.xticks(cluster_range)
optimal_k = cluster_range[np.argmax(scores)]
plt.axvline(x=optimal_k, color='red', linestyle='--', alpha=0.7, 
           label=f'Optimal: {optimal_k} clusters')
plt.legend()
plt.show()

print(f"\n🎯 Based on silhouette analysis, optimal number of clusters: {optimal_k}")

## Summary & Course Preview

### What We've Learned Today
- **Machine Learning can automatically discover patterns** in aerospace data that traditional methods might miss
- **Unsupervised learning** (clustering) requires no labeled data but can reveal hidden structure
- **Feature selection and preprocessing** are crucial for ML success in aerospace applications
- **Visualization and validation** help interpret ML results for domain experts

### Coming Up in AERO 689
- **Week 2**: Linear regression for drag prediction from wind tunnel data
- **Week 3**: Cross-validation and regularization to prevent overfitting
- **Week 4**: Classification algorithms for system fault detection
- **Weeks 9-12**: Deep learning for satellite imagery and physics-informed models

### Homework Assignment
Apply this analysis to a different aerospace dataset and write a 2-3 page report on your findings.

---

*"The goal is not to replace aerospace engineers with machine learning, but to make aerospace engineers more powerful with machine learning."*

**Next Class**: Linear Regression for Aircraft Performance Modeling